Generate gpt-4o-mini result

In [1]:
from datetime import datetime, timedelta
import pandas as pd

question_df = pd.read_json('eval/questions.json', lines=True)

def time_handler(time_interval:str):
    if time_interval=="Now":
        return datetime.strptime("2025/1/21 12:00", "%Y/%m/%d %H:%M")
    elif time_interval=="A few hours before":
        return datetime.strptime("2025/1/20 08:00", "%Y/%m/%d %H:%M")
    elif time_interval=="A few days before":
        return datetime.strptime("2025/1/15 12:00", "%Y/%m/%d %H:%M")
    elif time_interval=="A few weeks before":
        return datetime.strptime("2024/12/16 12:00", "%Y/%m/%d %H:%M")
    elif time_interval=="A few months before":
        return datetime.strptime("2024/9/22 12:00", "%Y/%m/%d %H:%M")
    elif time_interval=="A couple of years before":
        return datetime.strptime("2022/10/06 12:00", "%Y/%m/%d %H:%M")

for row in range(len(question_df)):
    for j, col in enumerate(['first_session_dialogue', 'second_session_dialogue', 'third_session_dialogue', 'fourth_session_dialogue', 'fifth_session_dialogue']):
        dialog = []
        current_time = time_handler(question_df['time_interval'][row][j])
        for i in range(int(len(question_df[col][row])/2)):
            current_time = current_time + timedelta(minutes=1)
            log = {
                "user":question_df[col][row][i*2],
                "assistant":question_df[col][row][i*2+1],
                "time":current_time.strftime("%Y-%m-%dT%H:%M:%SZ")
            }
            dialog.append(log)
        question_df.loc[row, col] = str(dialog)
question_df = question_df.drop(columns=['time_interval', 'double_check', 'reason', 'relationship'], axis=1)
# question_df

In [2]:
from long_memory.component import WeaviateLongMemory

long_mem = WeaviateLongMemory(user='gpt', model='gpt-4o-mini')

Detect existed gpt user group memory space, loading...
Detect existed gpt user child memory space, loading...


In [3]:
index = ['long_mem_result', 'long_mem_recall_result', 'grouping_try']
for i in index:
    question_df[i]=None

In [4]:
for row in range(500):
    print(f'==={row}===')
    
    question = f"user:{eval(question_df['question'][row])['user']}"
            
    # long mem process
    success = False
    while not success:
        try:
            long_mem.del_memory()
            for col in ['first_session_dialogue', 'second_session_dialogue', 'third_session_dialogue', 'fourth_session_dialogue', 'fifth_session_dialogue']:
                conv = eval(question_df[col][row])
                chat_logs = []
                for log in conv:
                    chat_logs.append({
                        "text":f"user:{log['user']}, assistant:{log['assistant']}",
                        "time":log['time']
                    })
                long_mem.add_chat_logs(chat_logs)
                if question_df.loc[row, 'grouping_try'] == None:
                    question_df.loc[row, 'grouping_try'] = long_mem.classify_chat_logs_try
                else: 
                    question_df.loc[row, 'grouping_try'] += long_mem.classify_chat_logs_try
            question_df.loc[row, 'grouping_try'] = question_df.loc[row, 'grouping_try']/5
            
            searched_memory = long_mem.get_memory(question, recall=False)
            question_df.loc[row, 'long_mem_result'] = str(searched_memory)
            # (recall)
            searched_memory = long_mem.get_memory(question, recall=True)
            question_df.loc[row, 'long_mem_recall_result'] = str(searched_memory)
            success = True
        except Exception as e:
            print(f"---long memory error:{e}, retry...---")
    
    if (row+1)%10==0:
        question_df.to_json("gpt_result.json", orient="records", lines=True)
question_df.to_json("gpt_result.json", orient="records", lines=True)

===0===
Detect empty group memory, create memory space...
Detect empty child memory, create memory space...
Save chat logs to long memory done.
Save chat logs to long memory done.
Save chat logs to long memory done.
Save chat logs to long memory done.
Save chat logs to long memory done.
===1===
Detect empty group memory, create memory space...
Detect empty child memory, create memory space...
Save chat logs to long memory done.
Save chat logs to long memory done.
Save chat logs to long memory done.
Save chat logs to long memory done.
Save chat logs to long memory done.
===2===
Detect empty group memory, create memory space...
Detect empty child memory, create memory space...
Save chat logs to long memory done.
Save chat logs to long memory done.
Save chat logs to long memory done.
Save chat logs to long memory done.
Save chat logs to long memory done.
===3===
Detect empty group memory, create memory space...
Detect empty child memory, create memory space...
Save chat logs to long memor

Score

In [11]:
question_df['long_score'] = None
question_df['long_recall_score'] = None

origin_df = pd.read_json('eval/eval_result_11_29.json', lines=True)

In [6]:
from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import re

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def llm_create(prompt):
        messages = [{"role": "user", "content": prompt}]
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
        )
        return completion.choices[0].message.content
    
def llm_response_handler(response:str):
    """handle llm response format, especially for llama family"""
    try:
        return json.loads(response)
    except:
        response = response.strip()
        try:
            return json.loads(re.search(r"```json(.*?)```", response, re.DOTALL).group(1).strip())
        except:
            try:
                return json.loads(re.search(r"```(.*?)```", response, re.DOTALL).group(1).strip())
            except:
                return response

In [7]:
evaluation_prompt = """You will act as a reviewer to score the following returned memory based on their relevance, conciseness and readability.
The fewer and more relevant the key messages are, the higher the score will be.
Here is the question and answer:{qa_pair}

The score range is from 1 to 10, with 1 being the worst and 10 being the best. Please refer to the following three examples for guidance:
A_system:{a_system}, score:{a_score}
B_system:{b_system}, score:{b_score}
C_system:{c_system}, score:{c_score}

Next, please score the returned memory for the following systems:
Long_memory_system:{long_memory_system}

Long_recall_memory_system:{long_recall_memory_system}

Please provide the scores in the following format:
```json
{{
    "long_memory_system":"1~10",
    "long_recall_memory_system":"1~10"
}}
```"""

In [8]:
for row in range(500):
    print(f'==={row}===')
    
    qa_pair = question_df['question'][row]
    
    short_searched = origin_df.loc[row, 'short_mem_result']
    short_score = origin_df.loc[row, 'short_score']
    base_dialog_searched = origin_df.loc[row, 'base_dialog']
    base_dialog_score = origin_df.loc[row, 'base_dialog_score']
    base_paragraph_searched = origin_df.loc[row, 'base_paragraph']
    base_paragraph_score = origin_df.loc[row, 'base_paragraph_score']
    
    long_searched = question_df.loc[row, 'long_mem_result']
    long_recall_searched = question_df.loc[row, 'long_mem_recall_result']
    
    p = evaluation_prompt.format(qa_pair=qa_pair, a_system=short_searched, a_score=short_score, b_system=base_dialog_searched, b_score=base_dialog_score, c_system=base_paragraph_searched, c_score=base_paragraph_score, long_memory_system=long_searched, long_recall_memory_system=long_recall_searched)
    res = llm_create(p)
    res_dict = llm_response_handler(res)
    
    question_df.loc[row, 'long_score'] = res_dict.get('long_memory_system')
    question_df.loc[row, 'long_recall_score'] = res_dict.get('long_recall_memory_system')
    # print(res_dict.get('reason'))
    
    if row%50==0:
        question_df.to_json("eval_result_gpt.json", orient="records", lines=True)
question_df.to_json("eval_result_gpt.json", orient="records", lines=True)

===0===
===1===
===2===
===3===
===4===
===5===
===6===
===7===
===8===
===9===
===10===
===11===
===12===
===13===
===14===
===15===
===16===
===17===
===18===
===19===
===20===
===21===
===22===
===23===
===24===
===25===
===26===
===27===
===28===
===29===
===30===
===31===
===32===
===33===
===34===
===35===
===36===
===37===
===38===
===39===
===40===
===41===


===42===
===43===
===44===
===45===
===46===
===47===
===48===
===49===
===50===
===51===
===52===
===53===
===54===
===55===
===56===
===57===
===58===
===59===
===60===
===61===
===62===
===63===
===64===
===65===
===66===
===67===
===68===
===69===
===70===
===71===
===72===
===73===
===74===
===75===
===76===
===77===
===78===
===79===
===80===
===81===
===82===
===83===
===84===
===85===
===86===
===87===
===88===
===89===
===90===
===91===
===92===
===93===
===94===
===95===
===96===
===97===
===98===
===99===
===100===
===101===
===102===
===103===
===104===
===105===
===106===
===107===
===108===
===109===
===110===
===111===
===112===
===113===
===114===
===115===
===116===
===117===
===118===
===119===
===120===
===121===
===122===
===123===
===124===
===125===
===126===
===127===
===128===
===129===
===130===
===131===
===132===
===133===
===134===
===135===
===136===
===137===
===138===
===139===
===140===
===141===
===142===
===143===
===144===
===145===
===146===
===147==

In [12]:
question_df.to_json("eval_result_gpt.json", orient="records", lines=True)

In [10]:
number=500
observe_df = question_df.drop(['first_session_dialogue', 'second_session_dialogue', 'third_session_dialogue', 'fourth_session_dialogue', 'fifth_session_dialogue'], axis=1)[:number]
print(observe_df['long_score'].astype(int).sum()/number)
print(observe_df['long_recall_score'].astype(int).sum()/number)

7.974
8.608
